## Goals

Given the images, text, and audio files, as well as machine learning outputs, your goal is to identify, use data analytics to answer the questions below. Your tasks is to use visual analytics to improve and understand machine learning outputs and apply visual analytics to track provenance, uncertainty, and confidence in machine learning results. Ultimately, you must link multiple data types to identify the group CGCS is seeking.

    1. Examine the outputs from the model – either from the detection results provided or the results from a model you chose. Which objects were identified well by the model and which were not?

    2. Demonstrate your process for using visual analytics to correct for classification errors in the results. How do you represent confidence and uncertainty? How could the correction process be made more efficient?

    3. Characterize the distribution of objects across the forty people.

        a. Which people have which objects?

        b. Identify groups of people that have object(s) in common.

    4. Which group do you think is the most likely group with the “totem”? What is your rationale for that assessment?

    5. Process question: Did you choose to use the object recognition model results provided or use your own machine learning algorithm? Why did you make that choice? What was the biggest challenge you faced?